<a href="https://colab.research.google.com/github/WaliMuhammadAhmad/Llama_2_FineTuning/blob/main/Llama_2_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7

In [2]:
!pip install transformers==4.31.0

In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    TrainerCallback,
    Trainer
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [4]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-hf"

# Fine-tuned model name
new_model = "Llama-2-7b-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64
# i recommend use
#lora_r = 16
# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1
# comment the dropout if lora_r = 16

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "/kaggle/working/"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True # must
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 2

# Batch size per GPU for evaluation
per_device_eval_batch_size = 2

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 100

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU
device_map = {"": 0}

In [5]:
!git config --global credential.helper store

In [6]:
from huggingface_hub import login

In [7]:
login(userdata.get('HF_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model from previous checkpoint
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()
"trainable params: 667,493 || all params: 86,543,818 || trainable%: 0.7712775047664294"

trainable params: 33,554,432 || all params: 3,533,967,360 || trainable%: 0.9494833591219133


'trainable params: 667,493 || all params: 86,543,818 || trainable%: 0.7712775047664294'

In [10]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (

In [11]:
checkpoint = torch.load('/content/checkpoint_1.pt')

In [12]:
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [13]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (

In [15]:
dataset_name = "dataset"
dataset = load_dataset(dataset_name, split="train")
dataset.shape

Resolving data files:   0%|          | 0/7440 [00:00<?, ?it/s]

(623113, 1)

In [16]:
test_dataset = dataset.shuffle().select(range(2000))    # selecting 2000 random instance from dataset, this is not fessible but just to make an idea how things works

In [17]:
test_dataset[0]

{'text': '<s> <<SYS>>Genrate Unit Test Case for this Unit <</SYS>>[INST] GlobAlbedoMosaicProductReader extends AbstractProductReader { String getMosaicFileRegex(String filename) { return pattern.matcher(filename).replaceFirst("h\\\\\\\\d\\\\\\\\dv\\\\\\\\d\\\\\\\\d"); } protected  GlobAlbedoMosaicProductReader(GlobAlbedoMosaicReaderPlugIn readerPlugIn); static File[] getModisPriorTileDirectories(String rootDirString); static File[] getAdamTileDirectories(String rootDirString); static File[] getPriorTileDirectories(String rootDirString, int stage); void setMosaicModisPriors(boolean mosaicModisPriors); void setMosaicNewModisPriors(boolean mosaicNewModisPriors); void setMosaicAdam(boolean mosaicAdam); void setMosaicAdamPriors(boolean mosaicAdamPriors); void setPriorStage(int adamPriorStage); @Override void close();  } [/INST] @Test public void testGetMosaicFileRegex() { String dimRegex = mosaicGrid.getMosaicFileRegex("GlobAlbedo.2005129.h18v04.dim"); assertEquals("GlobAlbedo.2005129.h\\\\

In [18]:
# using the LLaMA tokenizer
tokenizer("i am dumb")

{'input_ids': [1, 474, 626, 270, 3774], 'attention_mask': [1, 1, 1, 1, 1]}

In [19]:
test_dataset.to_pandas()['text'].apply(lambda x: tokenizer(x))[0]

{'input_ids': [1, 1, 29871, 3532, 14816, 29903, 6778, 15462, 10492, 13223, 4321, 11733, 363, 445, 13223, 529, 829, 14816, 29903, 6778, 29961, 25580, 29962, 402, 2127, 2499, 2580, 29877, 29924, 3628, 293, 7566, 6982, 4988, 25513, 7566, 6982, 426, 1714, 679, 29924, 3628, 293, 2283, 4597, 735, 29898, 1231, 10422, 29897, 426, 736, 4766, 29889, 4352, 261, 29898, 9507, 467, 6506, 6730, 703, 29882, 1966, 1966, 29881, 1966, 1966, 29881, 29894, 1966, 1966, 29881, 1966, 1966, 29881, 1496, 500, 6364, 29871, 402, 2127, 2499, 2580, 29877, 29924, 3628, 293, 7566, 6982, 29898, 29954, 2127, 2499, 2580, 29877, 29924, 3628, 293, 6982, 3247, 688, 797, 9591, 3247, 688, 797, 416, 2294, 3497, 2636, 679, 2111, 275, 29925, 13479, 29911, 488, 17392, 3842, 29898, 1231, 3876, 9170, 1231, 416, 2294, 3497, 2636, 679, 3253, 314, 29911, 488, 17392, 3842, 29898, 1231, 3876, 9170, 1231, 416, 2294, 3497, 2636, 679, 29925, 13479, 29911, 488, 17392, 3842, 29898, 1231, 3876, 9170, 1231, 29892, 938, 7408, 416, 1780, 731, 2

In [20]:
tokenized_dataset = test_dataset.map(lambda x: tokenizer(x["text"]))

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [21]:
# Define Training Arguments
training_args = TrainingArguments(
    output_dir="/results",
    per_device_eval_batch_size=8,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
)

# Predict the dataset
# we need tokenized dataset features required: ['labels', 'input_ids', 'attention_mask']
predictions = trainer.predict(tokenized_dataset)

# Output predictions
print(predictions)

ValueError: expected sequence of length 432 at dim 1 (got 921)

In [22]:
from transformers import DataCollatorWithPadding

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = test_dataset.map(tokenize_function, batched=True)

# Convert the tokenized dataset to a pandas DataFrame
tokenized_dataset_df = tokenized_dataset.to_pandas()

# Display the DataFrame
print(tokenized_dataset_df.head())

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

                                                text  \
0  <s> <<SYS>>Genrate Unit Test Case for this Uni...   
1  <s> <<SYS>>Genrate Unit Test Case for this Uni...   
2  <s> <<SYS>>Genrate Unit Test Case for this Uni...   
3  <s> <<SYS>>Genrate Unit Test Case for this Uni...   
4  <s> <<SYS>>Genrate Unit Test Case for this Uni...   

                                           input_ids  \
0  [1, 1, 29871, 3532, 14816, 29903, 6778, 15462,...   
1  [1, 1, 29871, 3532, 14816, 29903, 6778, 15462,...   
2  [1, 1, 29871, 3532, 14816, 29903, 6778, 15462,...   
3  [1, 1, 29871, 3532, 14816, 29903, 6778, 15462,...   
4  [1, 1, 29871, 3532, 14816, 29903, 6778, 15462,...   

                                      attention_mask  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
2  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
4  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..

In [24]:

# Prepare data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="/results",
    per_device_eval_batch_size=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
)

# Prepare tokenized dataset with necessary features
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask"])

# Predict the dataset
predictions = trainer.predict(tokenized_dataset)

# Output predictions
print(predictions)

ValueError: Column name ['text'] not in the dataset. Current columns in the dataset: ['input_ids', 'attention_mask']

In [ ]:
predictions.argmax(-1)
# returns which token has the highest probability

It rlly sucks to leave the work unfinished, but i am a grad student and still learning things. idk when i will finish this notebook